In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [3]:
# %load_ext autoreload
# %autoreload 2

In [10]:
from lincoln.layers import Dense
from lincoln.losses import LogSoftmaxLoss, LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.operations.activations import Sigmoid, LogSigmoid
from lincoln.network import NeuralNetwork

## Regression network

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

## Regression models

In [ ]:
from lincoln.losses import MeanSquaredError
from lincoln.optimizers import SGD

from lincoln.activations import LinearAct, Sigmoid

#### Linear Regression

In [ ]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

lr.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=82618,
       single_output=True);

In [ ]:
def mae(y_true, y_pred):
    return round(torch.mean(torch.abs(y_true - y_pred)).item(), 4)

def mse(y_true, y_pred):
    return round(torch.mean(torch.pow(y_true - y_pred, 2)).item(), 4)

In [ ]:
def eval_regression_model(lr: NeuralNetwork, 
                          X_test: Tensor, 
                          y_test: Tensor):
    preds = lr.forward(X_test)
    preds = preds.reshape(preds.shape[0])
    return mae(preds, y_test), mse(preds, y_test)

In [ ]:
eval_regression_model(lr, X_test, y_test)

Works!

#### "Neural Network" Regression

In [ ]:
nn = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

nn.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
           single_output=True);

In [ ]:
eval_regression_model(nn, X_test, y_test)

Works!

#### "Deep Learning" Regression

In [ ]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

dl.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
           single_output=True);

In [ ]:
eval_regression_model(dl, X_test, y_test)

Works!

## Classification models

#### Data prep (SciKit Learn)

In [ ]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [ ]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

#### "`LogSigmoid`" Logistic Regression

In [ ]:
logr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LogSigmoid())],
            loss = LogSigmoidLoss(),
            optimizer = SGD())

logr.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
           single_output=True);

In [ ]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
            optimizer = SGD())

logr_mse.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
           single_output=True);

In [ ]:
def accuracy(y_true, y_pred):
    return round(torch.sum(torch.eq(y_true, y_pred)).item() / y_true.size()[0], 4)

In [ ]:
def eval_classification_model(model: NeuralNetwork, 
                              X_test: Tensor, 
                              y_test: Tensor,
                              log_probs: bool = False):
    preds = model.forward(X_test)
    if log_probs:
        preds = torch.exp(preds)
    preds = preds > 0.5
    preds = preds.reshape(preds.shape[0]).type(torch.FloatTensor)  
    return accuracy(preds, y_test)

In [ ]:
eval_classification_model(logr, X_test, y_test, log_probs=True)

In [ ]:
eval_classification_model(logr_mse, X_test, y_test)

`LogSoftmaxLoss` works!

#### "Neural Network" Logistic Regression

In [ ]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
            optimizer = SGD())

logr_nn.fit(X_train, y_train, X_test, y_test,  
       epochs = 500,
       eval_every = 50,
       seed=82618,
           single_output=True);

In [ ]:
eval_classification_model(logr_nn, X_test, y_test)

**Works!**

## Convolution Demos

**Note:** Old losses and layers code.

In [ ]:
from lincoln.layers import Conv2D
from lincoln.operations import ReLU

In [ ]:
from torchvision.datasets import MNIST
mnist_trainset = MNIST(root="./explanatory/data", train=True, download=False, transform=None)
mnist_testset = MNIST(root="./explanatory/data", train=False, download=True, transform=None)

#### One hot encode labels

In [ ]:
data = mnist_trainset
num_labels = len(data.train_labels)
train_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    train_labels[i][data.train_labels[i]] = 1
train_labels.shape

In [ ]:
data = mnist_testset
num_labels = len(data.test_labels)
test_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    test_labels[i][data.test_labels[i]] = 1
test_labels.shape

In [ ]:
mnist_train = mnist_trainset.train_data.type(torch.float32).unsqueeze(3) / 255.0
mnist_test = mnist_testset.test_data.type(torch.float32).unsqueeze(3) / 255.0

In [ ]:
X_train = mnist_train
X_test = mnist_test
y_train = train_labels
y_test = test_labels

In [ ]:
def _conv_transform(input_: Tensor) -> Tensor:

    return input_.view(input_.shape[0],
                       input_.shape[1] * input_.shape[2] * input_.shape[3])

X_train_flat = _conv_transform(X_train)
X_test_flat = _conv_transform(X_test)

In [ ]:
conv_nn_ll = NeuralNetwork(
    layers=[Dense(neurons=250, 
                  activation=Sigmoid()),
            Dense(neurons=50, 
                  activation=Sigmoid()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_ll.fit(X_train_flat, y_train, X_test_flat, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

In [ ]:
def eval_nn(nn, X_test, y_test):
    test_preds = nn.forward(X_test)
    
    test_preds_l = torch.argmax(test_preds, dim=1)
    
    test_vals_l = torch.argmax(y_test, dim=1)
    
    return torch.sum(test_preds_l == test_vals_l)

In [ ]:
eval_nn(conv_nn_ll, X_test_flat, y_test)

In [ ]:
X_train_perm = X_train.permute(0, 3, 1, 2)
X_test_perm = X_test.permute(0, 3, 1, 2)

In [ ]:
# settings
pytorch = True

In [ ]:
conv_nn_logloss = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_logloss.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

In [ ]:
eval_nn(conv_nn_logloss, X_test_perm, y_test)

### Testing network without PyTorch layers 

In [ ]:
# %load_ext autoreload
# %autoreload 2

#### No Cython

In [ ]:
pytorch = False
cython = False

In [ ]:
conv_nn_normal = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_normal.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=1,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

#### Cython

In [ ]:
pytorch = False
cython = True

In [ ]:
conv_nn_cython = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_cython.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=1,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")